In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from unityagents import UnityEnvironment

from dqn_agent import DQN_AGENT

%matplotlib inline
from IPython import display
import time

In [ ]:
env = UnityEnvironment(file_name="VisualBanana_Linux/Banana.x86_64")

In [ ]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
def save_agent(agent, episode_count):
    checkpoint = {
        "action_size" : agent.action_size,
        "state_size"  : agent.state_size,
        "seed"        : agent.seed,
        "time_steps"  : agent.time_steps,
        "state_dict"  : agent.learning_network.state_dict(),
        "episodes"    : episode_count
    }
    filename = f"agent_{episode_count}.pth"
    torch.save(checkpoint, filename)
    return filename

In [ ]:
def plot_scores(scores, mean_scores, save_fig=False, filename=None):
    plt.plot(scores, label="Score")
    plt.plot(mean_scores, label="Rolling Mean")
    plt.xlabel("Episode")
    plt.ylabel("Score")
    plt.legend()
    if save_fig:
        plt.savefig(filename+'.png')
    plt.show()

In [ ]:
def initialize_experiences(agent, env, action_size, brain_name, size=50000):
    e_count = 0
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.visual_observations[0]
    
    while e_count < size:
        action = np.random.choice(np.arange(action_size))
        env_info = env.step(action)[brain_name]
        next_state = env_info.visual_observations[0]
        reward = env_info.rewards[0]
        done = env_info.local_done[0]
        agent.step(state, action, reward, next_state, done, train=False)
        state = next_state
        e_count += 1
        
        if done:
            env_info = env.reset(train_mode=True)[brain_name]
            state = env_info.visual_observations[0]            

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.visual_observations[0]
score = 0
scores = list()
mean_scores_window = list()
scores_window = deque(maxlen=100)

env_state_size = state.shape[-1]
env_action_size = brain.vector_action_space_size

agent = DQN_AGENT(env_state_size, env_action_size, 42)
eps_start = 1.0
eps_decay = 0.995
eps_fixed = 0.01
eps = eps_start

start_time = time.time()
print("Initializing Replay Memory [50,000 Experiences]...")
# initialize_experiences(agent, env, env_action_size, brain_name)
print("Finished Initializing Replay Memory ...\tTime : {:.1f} secs".format(time.time() - start_time))

start_time = time.time()

for i in range(1, 1000):
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.visual_observations[0]
    state = state.transpose(0, 3, 1, 2)
    score = 0
    
    while True:
        action = agent.act(state, eps)
        env_info = env.step(action)[brain_name]
        next_state = env_info.visual_observations[0]
        next_state = next_state.transpose(0, 3, 1, 2)
        reward = env_info.rewards[0]
        done = env_info.local_done[0]
        agent.step(state, action, reward, next_state, done)
        score += reward
        state = next_state

        if done:
            scores.append(score)
            scores_window.append(score)
            mean_scores_window.append(np.mean(scores_window))
            break
        
    eps = max(eps_fixed, eps*eps_decay)
    
    if i % 10 == 0:
        display.clear_output(wait=True)
        print("EPISODE {} -->\tScore : {}\tRolling Avg : {:.2f}\tMax : {:.2f}\tTime : {:.1f} secs\t eps : {:.2f}".format(i, score, np.mean(scores_window), np.max(scores), time.time()-start_time, eps))
        plot_scores(scores, mean_scores_window)
        start_time = time.time()
        
    if np.mean(scores_window) > 13.5:
        display.clear_output(wait=True)
        print(f"Hooray... Environment Solved in {i} episodes!! Avg. Score : {np.mean(scores_window)}")
        checkpoint = save_agent(agent, i)
        print(f"Checkpoint : {checkpoint}")
        plot_scores(scores, mean_scores_window, save_fig=True, filename=checkpoint.split('.')[0])
        break
        